# Семинар 2. Метрические методы классификации и регрессии.

Подключение библиотек. 
$
\newcommand{\R}{\mathbb{R}}
\newcommand{\X}{\mathbb{X}}
\newcommand{\norm}[1]{\lVert #1 \rVert}
\newcommand{\abs}[1]{\left| #1 \right|}
\newcommand{\E}{\mathbb{E}}
\renewcommand{\Prob}{\mathbb{P}}
\renewcommand{\le}{\leqslant}
\renewcommand{\ge}{\geqslant}
$

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as wg

from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder

# 1. Примеры расстояний

* Расстояние Минковского ($r \ge 1$): $$\rho(x, x') = \left(\sum_{k=1}^n \abs{x_k - x'_k}^r\right)^{1/r}.$$

* Евклидово расстояние ($r = 2$): $$\rho(x, x') = \left(\sum_{k=1}^n \abs{x_k - x'_k}^2\right)^{1/2}.$$

* Манхэттенское расстояние ($r = 1$): $$\rho(x, x') = \sum_{k=1}^n \abs{x_k - x'_k}.$$

* Расстояние Чебышёва ($r = +\infty$): $$\rho(x, x') = \max_{1 \le k \le n} \abs{x_k - x'_k}.$$

__Задача:__ Реализовать функцию, которая возвращает расстояние Минковского между двумя векторами.

* На вход:
    * вектор $x$;
    * вектор $y$;
    * параметр $0 < r < \infty$.
* На выход:
    * расстояние Минковского $r$ между векторами $x$ и $y$

In [ ]:
def distance(x, y, r=2):
    # написать код

Рисуем линии уровня для расстояний: 

In [ ]:
def plot_distance(r):
    N = 256
    X_list = np.linspace(-1, 1, N)
    Y_list = np.linspace(-1, 1, N)

    X = np.zeros((N, N))
    Y = np.zeros((N, N))
    Z = np.zeros((N, N))

    for i, x in enumerate(X_list):
        for k, y in enumerate(Y_list):
            X[i, k] = x
            Y[i, k] = y
            Z[i, k] = distance(np.array([x, y]), np.array([0, 0]), r)


    plt.figure(figsize=(5, 5))
    plt.contour(X, Y, Z, levels=32)
    plt.title(f'Расстояние Минковского, r={r}')
    plt.show()

In [ ]:
plot_distance(2)
plot_distance(1)
plot_distance(0.5)
plot_distance(20)

Немного [интерактива](https://ipywidgets.readthedocs.io/en/stable/):

In [ ]:
output = wg.Output()

def func(r):
    with output:
        output.clear_output(wait=True)
        plot_distance(r)

wg.interact(func,
            r=wg.FloatSlider(min=0.1, max=10, value=2.),
            continuous_update=False)
display(output)

## Любопытный факт.

Линии уровня на рисунке называются суперэллипсам: $$ \abs{\frac{x}{a}}^n + \abs{\frac{y}{b}}^n = 1$$

Пример использования: $$ \abs{\frac{x}{60}}^5 + \abs{\frac{y}{60}}^5 = 1$$ -- иконки в iOS! Называется сквиркл: square + circle. Подробнее в [источнике](https://droider.ru/post/iphone-eto-skvirkl-eto-chto-eshhyo-za-zver-16-08-2020/).

# 2. Классификация

## 2.1. Метод $k$ ближайших соседей

Обобщенный метрический классификатор: $$ a(u, \mathbb{X}) = \arg\max\limits_{y \in \mathbb{Y}} \sum\limits_{i=1}^{l} \left[y_{u}^{(i)} = y\right] \cdot w(i, u),$$
где $y_u^{(i)}$ -- $i$-ый сосед объекта $u \in \mathbb{X}$, $w(i, u)$ -- оценка степени важности $i$-го соседа для классификации объекта $u$.

__Метод $k$ ближайших соседей:__
    $$
        w(i, u) = [i \le k], \quad a(u, \X) = \arg\max_{y \in Y} \sum_{i=1}^{k}[y_u^{(i)} = y]
    $$

In [ ]:
frame = sns.load_dataset('iris')
frame['species_code'] = LabelEncoder().fit_transform(frame['species'])

print('Число записей:', frame.shape[0])
frame.head()

__Задача__ Построить диаграмму рассеяния для `sepal_length` и `petal_length`, раскрасить по цветам в зависимости от целевой переменной `species`.

In [ ]:
# код

Построим классификатор kNN для определения типа ириса:

In [ ]:
X = frame[['sepal_length', 'petal_length']].values
y = frame.species_code.values

In [ ]:
knn = KNeighborsClassifier(3)
knn.fit(X, y)
knn

In [ ]:
frame['prediction'] = knn.predict(X)
frame[frame.species_code != frame.prediction].head()

Нарисуем "карту" влияний.

In [ ]:
def plot_knn(knn, title='', xlabel='', ylabel=''):
    X = knn._fit_X
    y = knn._y
    
    x_low = X.min(axis=0)
    x_high = X.max(axis=0)
    x_d = x_high - x_low
        
    x_low -= 0.05 * x_d
    x_high += 0.05 * x_d
    
    X1 = np.linspace(x_low[0], x_high[0], 256)
    X2 = np.linspace(x_low[1], x_high[1], 256)
    
    X1, X2 = np.meshgrid(X1, X2)
    Z = np.column_stack([X1.flatten(), X2.flatten()])
    Z = knn.predict(Z)
    Z = Z.reshape(X1.shape)
    
    plt.figure(figsize=(7, 7))
    plt.contourf(X1, X2, Z, cmap='brg', levels=3, alpha=0.6)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='brg')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

Метод одного ближайшего соседа:

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X, y)
plot_knn(knn, 'KNN(1)', 'sepal_length', 'petal_length')

In [ ]:
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X, y)
plot_knn(knn, 'KNN(2)', 'sepal_length', 'petal_length')

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X, y)
plot_knn(knn, 'KNN(5)', 'sepal_length', 'petal_length')

In [ ]:
knn = KNeighborsClassifier(n_neighbors=30)
knn.fit(X, y)
plot_knn(knn, 'KNN(30)', 'sepal_length', 'petal_length')

И снова интерактив:

In [ ]:
output = wg.Output()

def func(k):
    with output:
        output.clear_output(wait=True)
        knn = KNeighborsClassifier(k)
        knn.fit(X, y)
        plot_knn(knn, f'KNN({k})', 'sepal_length', 'petal_length')
    
wg.interact(func,
            k=wg.IntSlider(min=1, max=50, value=5),
            continuous_update=False)
display(output)

## 2.2. Взвешенный метод ближайших соседей

Метод $k$ __взвешенных__ ближайших соседей:
    $$w(i, u) = [i \le k] \cdot w_i, \quad a(u, \X) = \arg\max_{y \in Y} \sum_{i=1}^{k}[y_u^{(i)} = y] w_i.$$
    
Возможные подходы:

* Линейно убывающие веса: $$ w_i = \frac{k + 1 - i}{k}. $$
* Экспоненциально убывающие веса: $$ w_i = q^i, \quad 0 < q < 1. $$

__Задача:__ Реализовать функцию экспоненциальных весов.
* На вход: 
    * вектор расстояний длины $k$ (которые мы использовать не будем);
    * параметр $q$, значение по умолчанию $q=0.9$.
* На выход:
    * массив длины $k$ с весами $q^i, 1 \le i \le k$.

In [ ]:
def exp_weights_func(distances, q=0.9):
    # код

In [ ]:
indices = np.arange(50)
weights = exp_weights_func(indices)

plt.figure(figsize=(15, 4))
plt.bar(indices, weights)
plt.title('Экспоненциально убывающие веса')
plt.xlabel('Номер соседа')
plt.ylabel('Вес')
plt.show()

In [ ]:
def exp_weights(distances):
    return np.array([exp_weights_func(row) for row in distances])

knn = KNeighborsClassifier(5, weights=exp_weights)
knn.fit(X, y)
knn.predict(X)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7, weights=exp_weights)
knn.fit(X, y)
plot_knn(knn, 'KNN(7, weights=exp(0.9)', 'sepal_length', 'petal_length')

In [ ]:
# обертка
def exp_weights(q):
    def func(d):
        return np.array([exp_weights_func(row, q) for row in d])
    return func

Интерактивчик:

In [ ]:
output = wg.Output()

def func(k, q):
    with output:
        output.clear_output(wait=True)
        knn = KNeighborsClassifier(k, weights=exp_weights(q))
        knn.fit(X, y)
        plot_knn(knn, f'KNN({k}, weights=exp({q}))', 'sepal_length', 'petal_length')
    
wg.interact(func,
            k=wg.IntSlider(min=1, max=50, value=5),
            q=wg.FloatSlider(min=0.01, max=1.0, step=0.01, value=1.0),
            continuous_update=False)
display(output)

График весов:

In [ ]:
output = wg.Output()

def func(q):
    with output:
        output.clear_output(wait=True)
        
        indices = np.arange(50)
        weights = exp_weights_func(indices, q)

        plt.figure(figsize=(15, 4))
        plt.bar(indices, weights)
        plt.title(f'Экспоненциально убывающие веса, q={q}')
        plt.xlabel('Номер соседа')
        plt.ylabel('Вес')
        plt.show()
    
wg.interact(func,
            q=wg.FloatSlider(min=0.01, max=1.0, step=0.01, value=1.0),
            continuous_update=False)
display(output)

# 3. Регрессия

## 3.1. Метод $k$ ближайших соседей

* Классический: $$ w(i, u) = [i \le k], $$ $$ a(u, \X) = \frac{1}{k} \cdot \sum_{i=1}^k y_u^{(i)}. $$
* Общий вид (взвешенное среднее): $$a(u, \X) = \frac{\sum\limits_{i=1}^{\ell} w(i, u) \cdot  y_u^{(i)}}{\sum\limits_{i = 1}^{\ell} w(i, u)}.$$

Возможные подходы:

* Линейно убывающие веса: $$ w_i = \frac{k + 1 - i}{k}. $$
* Обратное расстояние: $$ w(i, u) = \frac{1}{\rho(u, x_u^{(i)})}. $$
* Экспоненциально убывающие веса: $$ w_i = q^i, \quad 0 < q < 1. $$

In [ ]:
N = 48

def real_func(x):
    return np.sin(x) / x

X = np.linspace(-2 * np.pi, 2 * np.pi, N)
y = real_func(X) + np.random.normal(0, 0.2, size=N)

print('Размер X:', X.shape)
print('Размер y:', y.shape)

x_plt = np.linspace(-2 * np.pi, 2 * np.pi, 1024)
y_plt = real_func(x_plt)

plt.figure(figsize=(15, 4))
plt.scatter(X, y, color='blue')
plt.plot(x_plt, y_plt, color='green', linestyle='--')
plt.show()

Учим KNN:

In [ ]:
knn = KNeighborsRegressor(n_neighbors=1)
knn.fit(X.reshape(-1, 1), y)

In [ ]:
def plot_knn_reg(knn, title='', xlabel='X', ylabel='y'):
    x_plt = np.linspace(-2 * np.pi, 2 * np.pi, 1024)
    knn_plt = knn.predict(x_plt.reshape(-1, 1))
    
    plt.figure(figsize=(15, 4))
    plt.scatter(X, y, color='blue')
    plt.plot(x_plt, knn_plt, color='blue', lw=1)
    plt.plot(x_plt, y_plt, color='green', linestyle='--')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

In [ ]:
plot_knn_reg(knn, f'KNN(1)')

In [ ]:
output = wg.Output()

def func(k):
    with output:
        output.clear_output(wait=True)
        knn = KNeighborsRegressor(k, )
        knn.fit(X.reshape(-1, 1), y)
        plot_knn_reg(knn, f'KNN({k})')
    
wg.interact(func,
            k=wg.IntSlider(min=1, max=N, value=5),
            continuous_update=False)
display(output)

## 3.2. Метод $k$ взвешенных ближайших соседей

__Задача:__ реализовать функцию, возвращающую обратное расстояние.
* На входе:
    * массив расстояний distances;
* На выходе:
    * массив соответствующих весов для них.
    
__Предупреждение:__ Сумма весов должна быть равна 1.

In [ ]:
def inv_dist(distances):
    # код

Посмотрим, как это выглядит:

In [ ]:
output = wg.Output()

def func(k):
    with output:
        output.clear_output(wait=True)
        knn = KNeighborsRegressor(k, weights=inv_dist)
        knn.fit(X.reshape(-1, 1), y)
        plot_knn_reg(knn, f'KNN({k}, weights=inv_dist)')
    
wg.interact(func,
            k=wg.IntSlider(min=1, max=N, value=5),
            continuous_update=False)
display(output)

Посмотрим теперь экспонециальное взвешивание:

In [ ]:
def reg_exp_weights_func(distances, q=0.9):
    # код

In [ ]:
def reg_exp_weights(q):
    def func(d):
        return np.array([reg_exp_weights_func(row, q) for row in d])
    return func

In [ ]:
output = wg.Output()

def func(k, q):
    with output:
        output.clear_output(wait=True)
        knn = KNeighborsRegressor(k, weights=reg_exp_weights(q))
        knn.fit(X.reshape(-1, 1), y)
        plot_knn_reg(knn, f'KNN({k}, weights=exp({q}))')
    
wg.interact(func,
            k=wg.IntSlider(min=1, max=50, value=5),
            q=wg.FloatSlider(min=0.01, max=1.0, step=0.01, value=1.0),
            continuous_update=False)
display(output)

# 4. Упражнения

__Задача 1:__ Реализовать метод $k$ ближайших соседей для регрессии и классификации с нуля.

In [ ]:
class HandmadeKNNClassifier:
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
        
    # на вход: 
    # X -- матрица признаков, np.array, shape (N, d)
    # y -- массив ответов, np.array, shape (N,)
    # на выход:
    # self -- объект класса
    # этот метод уже реализован
    def fit(self, X, y):
        self._fit_X = X.copy()
        self._y = y.copy()
        return self
        
    # на вход: 
    # X -- матрица признаков, np.array, shape (N, d)
    # на выход:
    # y -- массив меток, np.array, shape (N,)
    def predict(self, X):
        # реализуйте этот метод
        return np.zeros(X.shape[0])

In [ ]:
# проверка
X = frame[['sepal_length', 'sepal_width']].values
y = frame['species_code'].values

k = 6

knn = HandmadeKNNClassifier(n_neighbors=k)
knn.fit(X, y)

plot_knn(knn, f'HandmadeKNN({k})', 'sepal_length', 'petal_length')

knn_gt = KNeighborsClassifier(n_neighbors=k)
knn_gt.fit(X, y)
if not (knn.predict(X) == knn_gt.predict(X)).min():
    print('Ответы не совпадают с KNeighborsClassifier из sklearn')
else:
    print('Успех!')

In [ ]:
class HandmadeKNNRegressor:
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
        
    # на вход: 
    # X -- матрица признаков, np.array, shape (N, d)
    # y -- массив ответов, np.array, shape (N,)
    # на выход:
    # self -- объект класса
    # этот метод уже реализован
    def fit(self, X, y):
        self._fit_X = X.copy()
        self._y = y.copy()
        return self
        
    # на вход: 
    # X -- матрица признаков, np.array, shape (N, d)
    # на выход:
    # y -- массив меток, np.array, shape (N,)
    def predict(self, X):
        # реализуйте этот метод
        return np.zeros(X.shape[0])

In [ ]:
# проверка
N = 48
def real_func(x):
    return np.sin(x) / x

X = np.linspace(-2 * np.pi, 2 * np.pi, N)
y = real_func(X) + np.random.normal(0, 0.2, size=N)
X = X.reshape(-1, 1)

k = 7

knn = HandmadeKNNRegressor(n_neighbors=k)
knn.fit(X, y)

plot_knn_reg(knn, f'HandmadeKNN({k})')

knn_gt = KNeighborsRegressor(n_neighbors=k)
knn_gt.fit(X, y)

if not (knn.predict(X) == knn_gt.predict(X)).min():
    print('Ответы не совпадают с KNeighborsRegressor из sklearn')
else:
    print('Успех!')